In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

Сетка по статье https://drive.google.com/file/d/1ia3W9vgdZLurEx4R7ndVL05clQLEarKF/view?usp=sharing . Оставлена только первая сверточная часть temporal_feature_extractor

In [30]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [31]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 100
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [32]:
def get_padding(in_size, kernel_size, stride):
    if (in_size % stride == 0):
        padding = max(kernel_size - stride, 0)
    else:
        padding = max(kernel_size - (in_size % stride), 0)
    return (padding)    

In [33]:
def get_temporal_feature_extractor(input_size):
  pad = get_padding(input_size, 5,  2)
  conv1 = nn.Conv3d(1, 32, kernel_size = (1, 1, 5), stride=(1, 1, 2), padding=(0, 0, 2))
  relu1 = nn.ReLU()
  pad = get_padding(input_size, 3,  2)
  conv2 = nn.Conv3d(32, 32, kernel_size = (1, 1, 3), stride=(1, 1, 2), padding=(0, 0, pad))
  relu2 = nn.ReLU()
  conv3 = nn.Conv3d(32, 32, kernel_size = (1, 1, 3), stride=(1, 1, 2), padding=(0, 0, pad))
  relu3 = nn.ReLU()
  conv4 = nn.Conv3d(32, 32, kernel_size = (1, 1, 16), stride=(1, 1, 16), padding=0)
  relu4 = nn.ReLU()
  #print("11")
  result = torch.nn.Sequential(conv1, relu1, conv2, relu2, conv3, relu3, conv4, relu4)
  #print("22")
  #print(result)
  return(result)

In [34]:
# def get_regional_feature_extractor():
#   conv1 = nn.Conv2d(32, 32, kernel_size = (3, 3), stride=1 , padding='same')
#   relu1 = nn.ReLU()
#   conv2 = nn.Conv2d(32, 32, kernel_size = (3, 3), stride=1 , padding='same')
#   relu2 = nn.ReLU()
#   return(torch.nn.Sequential(conv1, relu1, conv2, relu2))

In [35]:
# class Asymmetric_feature_extractor(torch.nn.Module): 
#    def __init__(self):
#      super().__init__()
#      self.conv = nn.Conv2d(32, 64, kernel_size = 1, stride=1 , padding='same')
#      self.relu = nn.ReLU()
#    def forward(self, input):
#      #input(bs, nf, h, w)
#      #print(input.shape)
#      half_mat = torch.split(input, (4, 1, 4), dim = 3)
#      #print(half_mat[0].shape)
#      #print(half_mat[2].shape)
#      input_new =  half_mat[0] -  half_mat[2]
#      #print(input.shape)
#      output = self.conv(input_new)
#      output = self.relu(output)
#      #print(output.shape)
#      return output

# def get_asymmetric_feature_extractor():
#     return (Asymmetric_feature_extractor())


In [36]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, nfeatures, ntimes_in_sample):
      super().__init__()
      #print("1")
      self.tfe = get_temporal_feature_extractor(ntimes_in_sample) #(bs, 1, h = 9, w = 9, s = 128) -> (bs, h = 9, w = 9, s = 1)
      #print("2")
      #self.rfe = get_regional_feature_extractor() #(h = 9, w = 9, s = 32) -> (h = 9, w = 9, s = 32)
      #print("3")
      #self.afe = get_asymmetric_feature_extractor()
      #print("4")
      self.flat1 = nn.Flatten(1, 3)
      #self.flat2 = nn.Flatten(1, 3)
      self.input_linear_size = 9 * 9 * 32#int(hcanals * (wcanals//2)* nfeatures * 2 + hcanals * (wcanals)* nfeatures)
      #print(self.input_linear_size)
      #print("5")
      self.fc1 = nn.Linear(self.input_linear_size, 20)
      self.relu1 = nn.ReLU()
      self.drop = nn.Dropout(0.3)
      self.fc2 = nn.Linear(20, 2)
   def forward(self, input):
      input = input.unsqueeze(1)
      #print(f"input_shape = {input.shape}")
      
      #input (bs, in_canals = 1,  h=9, w=9, s=128)
      output_tfe = self.tfe(input)
      #print(f"output_tfe.shape = {output_tfe.shape}")
      #output_tfe (bs, in_canals = 32,  h=9, w=9, s=1)
      output_tfe = output_tfe.squeeze(4)
      #print(f"output_tfe.shape = {output_tfe.shape}")
      #output_rfe (bs, canals = 32,  h=9, w=9)
      #output_rfe = self.rfe(output_tfe)
      #print(f"output_rfe.shape = {output_rfe.shape}")
      #output_afe = self.afe(output_tfe)
      #print(f"output_afe.shape = {output_afe.shape}")
      #output_rfe (bs, canals = 64,  h=9, w=4)

      #output_rfe = output_rfe.permute(0, 2, 3, 1) 
      #output_afe = output_afe.permute(0, 2, 3, 1) 
     # output_tfe = output_tfe.permute(0, 2, 3, 1) 
      output_tfe_flatten = self.flat1(output_tfe)
      #print(f"output_tfe.shape = {output_tfe.shape}")
      #print(f"output_rfe.shape = {output_rfe.shape}")
      #print(f"output_afe.shape = {output_afe.shape}")
      #output_rfe_flatten = self.flat1(output_rfe)
      #print(f"output_rfe_flatten.shape = {output_rfe_flatten.shape}")
      #output_afe_flatten = self.flat2(output_afe)
      #print(f"output_afe_flatten.shape = {output_afe_flatten.shape}")
      #output1 = self.drop(self.fc1(torch.cat((output_rfe_flatten, output_afe_flatten), dim = 1)))
      output1 = self.drop(self.fc1(output_tfe_flatten))
      #print(f"output1.shape = {output1.shape}")
      output1_relu = self.relu1(output1)
      #print(f"output1_relu.shape = {output1_relu.shape}")
      output2 = self.fc2(output1_relu)
      #print(f"output2.shape = {output2.shape}")
      return output2


In [37]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 4.5

In [38]:
import glob
import pickle
from collections import Counter

class EmotionDataset(Dataset):
    def __init__ (self, indexes, data, labels_bin):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       self.data = []
       self.labels = []
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       self.indexes = indexes
       self.len_files = []
       for sub in range(len(data)): #sub   - человек
            self.data.append(data[sub][indexes[sub], :, 3 * NTIMES_IN_SEC :  LEN_RECORD_IN_SECONDS * NTIMES_IN_SEC + 3 * NTIMES_IN_SEC])
            self.len_files.append(len(indexes[sub]) *  LEN_RECORD_IN_SECONDS - 1) #столько записей будет храниться для каждого человека - количство видео для человека на 60 сек
            self.len_record = LEN_RECORD_IN_SECONDS
            labels_bin_sub = (labels_bin[sub])
            self.labels.append(labels_bin_sub[indexes[sub]])          
            # счетчик числа 1 и 0 в датасете
            for type_emotion in range(4):
              self.cnt[type_emotion].update(list(self.labels[-1][:, type_emotion]))    

            
       self.len_cumsum = np.cumsum(self.len_files)     


    # общее число сеймплов по 1 сек в датасете. их будет 32 * количество видео в датасете * 60 
    def __len__(self):
        result =  sum(self.len_files) - 10
        return result


    # по номеру сеймпла вытягиваем номер человека номер видео и номер секунды
    def get_index_record(self, item):
      for sub in range(len(self.len_cumsum)):
         #print(item, self.len_cumsum[i_file])
         if (item > self.len_cumsum[sub]):
            continue
         else:
            break
      if sub == 0:
         index_in_file = item
      else:
         index_in_file = item  - self.len_cumsum[sub - 1]
      nvideo = index_in_file//(self.len_record)# * LEN_RECORD_IN_SECONDS *  NTIMES_IN_SEC)
      nsec = (index_in_file - nvideo * self.len_record) # *   NTIMES_IN_SEC)

      return sub, nvideo, nsec

   
    def __getitem__(self, item):
      sample = {}
      #print(item)
      sub, nvideo, nsec = self.get_index_record(item)
      #print(i_file, index_in_file, nvideo, nsec )
      sample['data'] = np.zeros((HCANALS, WCANALS, NTIMES_IN_SAMPLE))
      sample_from_one_canals = []
      for i_canal in range(NCANALS):
        sample_from_one_canal = self.data[sub][nvideo, i_canal, nsec * 128 : nsec * 128 + 128]
        sample_from_one_canals.append(sample_from_one_canal)
        #print(sample_from_one_canal.shape)
      sample_from_one_canals = np.asarray(sample_from_one_canals).copy()
      # здесь делаем нормализацию фактически по поверхности головы (по одному времени t для всех каналов)
      sample_from_one_canals = scipy.stats.zscore(sample_from_one_canals, axis = 0)
      for i_canal in range(NCANALS):
          sample['data'][electrode_matrix[list_electrodes[i_canal]][0],  electrode_matrix[list_electrodes[i_canal]][1]] = sample_from_one_canals[i_canal]
      sample['data'] = torch.FloatTensor(sample['data'])
      sample['labels']  = torch.LongTensor(self.labels[sub][nvideo])
      
      return sample

In [39]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model


In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()

In [41]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (tfe): Sequential(
    (0): Conv3d(1, 32, kernel_size=(1, 1, 5), stride=(1, 1, 2), padding=(0, 0, 2))
    (1): ReLU()
    (2): Conv3d(32, 32, kernel_size=(1, 1, 3), stride=(1, 1, 2), padding=(0, 0, 1))
    (3): ReLU()
    (4): Conv3d(32, 32, kernel_size=(1, 1, 3), stride=(1, 1, 2), padding=(0, 0, 1))
    (5): ReLU()
    (6): Conv3d(32, 32, kernel_size=(1, 1, 16), stride=(1, 1, 16))
    (7): ReLU()
  )
  (flat1): Flatten(start_dim=1, end_dim=3)
  (fc1): Linear(in_features=2592, out_features=20, bias=True)
  (relu1): ReLU()
  (drop): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=20, out_features=2, bias=True)
)

Считываем данные

In [42]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    #break

Отлаживаюсь на эмоции с индексом 0 (valence)

In [43]:
type_emotion = 0

Бинаризуем метки и разбиваем видео для каждого человека в соответствии с метками этого человека - оно не всегда будет сбалансированным, как оказалось, но хотя бы метки буду представлены наиболее равномерно. Выведем количестов меток 1 для каждого человека в разбиении

In [44]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score
k  = 5
labels_bin = []
for sub in range(32):
  temp = labels[sub] >= 4.5
  #print(labels[i])
  #print(temp)
  labels_bin.append(temp)
  print(sum(labels_bin[sub][:, type_emotion]), end=' ')

inds_train = []
inds_test = []
for sub in range(32):  
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    for ind_train, ind_test in  balanced_split:
        print(ind_train, ind_test)
        #количестов меток 1 в трейне и тесте
        print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train.append(ind_train)
        inds_test.append(ind_test)
    #break

20 29 30 17 28 33 30 26 23 24 27 23 20 20 22 17 26 30 26 28 25 22 32 19 24 27 33 25 23 28 29 22 [ 1  2  3  5  6  7  8  9 10 12 13 14 15 16 18 20 21 22 23 24 25 26 28 30
 31 32 33 35 36 37 38 39] [ 0  4 11 17 19 27 29 34]
16 4
[ True  True  True  True  True  True False False False False False False
  True False  True  True  True  True  True  True  True  True False False
 False False False False False False False  True] [ True  True False  True  True False False False]
[ 0  1  3  4  5  7  8  9 10 11 12 13 17 18 19 20 21 22 23 24 25 26 27 28
 29 30 31 34 35 36 37 38] [ 2  6 14 15 16 32 33 39]
16 4
[ True  True  True  True  True  True False False False False False False
  True  True  True  True  True  True  True  True  True  True False False
 False False False False False False False False] [ True  True False  True False False False  True]
[ 0  1  2  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 23 25 27 28
 29 31 32 33 34 35 38 39] [ 3  9 22 24 26 30 36 37]
16 4
[ True  True  True  Tr

In [45]:
train_dataset = EmotionDataset(inds_train, data, labels_bin)

class_weights_all = [1/train_dataset.cnt[0][i] for i in range(2)]
weights_samples =  [0] * train_dataset.__len__()

#A = np.zeros((32, 40, 60))

for i in range(train_dataset.__len__()):
    sub, nvideo, nsec = train_dataset.get_index_record(i)
    #A[sub, nvideo, nsec] = 1
    #print(train_dataset.labels[i_file][nvideo])
    weights_samples[i] = class_weights_all[int(train_dataset.labels[sub][nvideo, type_emotion])]

weighted_sampler = WeightedRandomSampler(
    weights=weights_samples,
    num_samples=len(weights_samples),
    replacement=True
)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=True, sampler=weighted_sampler)

# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                pin_memory=True, shuffle=True, drop_last=True)



val_dataset = EmotionDataset(inds_test, data, labels_bin)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)

In [46]:
sample = train_dataset.__getitem__(300)
print(sample['data'].shape)
print(sample['data'][:, :, 0])
print(sample['labels'])

torch.Size([9, 9, 128])
tensor([[ 0.0000,  0.0000,  0.0000, -0.9883,  0.0000, -0.5997,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000, -1.3634,  0.0000, -0.9291,  0.0000,  0.0000,
          0.0000],
        [-0.4436,  0.0000, -1.4477,  0.0000, -1.8068,  0.0000, -1.2050,  0.0000,
          0.3486],
        [ 0.0000, -0.2759,  0.0000, -1.2056,  0.0000, -1.6140,  0.0000, -0.1760,
          0.0000],
        [ 0.2685,  0.0000, -0.0652,  0.0000, -0.5555,  0.0000, -0.3704,  0.0000,
          0.3384],
        [ 0.0000,  0.8265,  0.0000,  0.7608,  0.0000,  0.0538,  0.0000,  0.0970,
          0.0000],
        [ 1.5165,  0.0000,  1.7218,  0.0000,  0.7796,  0.0000,  0.4185,  0.0000,
          0.4772],
        [ 0.0000,  0.0000,  0.0000,  2.2677,  0.0000,  0.3198,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.3978,  1.0465,  0.4072,  0.0000,  0.0000,
          0.0000]])
tensor([1, 0, 1, 1])


In [47]:
print(inds_train[sub])

[ 0  1  3  5  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25 26 27
 28 30 31 32 36 37 38 39]


In [48]:
 sub, nvideo, nsec = train_dataset.get_index_record(300)
 print(sub, nvideo, nsec)
 print(data[sub][inds_train[sub][nvideo], 0, 3 * 128 + nsec * 128])
 print(labels_bin[sub][inds_train[sub][nvideo], :])
 


0 5 0
-8.868155152570555
[ True False  True  True]


In [49]:
criterion = nn.CrossEntropyLoss(reduce = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


распределение положительных меток в датасете по каждой эмоции

In [50]:
print(train_dataset.cnt)
print(val_dataset.cnt)


[Counter({True: 639, False: 385}), Counter({True: 655, False: 369}), Counter({True: 678, False: 346}), Counter({True: 718, False: 306})]
[Counter({True: 169, False: 87}), Counter({True: 163, False: 93}), Counter({True: 178, False: 78}), Counter({True: 172, False: 84})]


In [51]:
def train(model, loader, criterion, optimizer, device, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):

            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs


In [52]:
def evaluate(model, loader, criterion, device):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            epoch_loss += loss.item() 
        
    return epoch_loss / s

In [53]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(tqdm(loader, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
         

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

            
        print(accuracy_score(real, pred)) 
        print(confusion_matrix(real, pred))  
        print(classification_report(real, pred))   
        return (accuracy_score(real, pred)) , (f1_score(real, pred, 'macro')) 
        #plt.hist(real)

In [54]:
args.epochs = 10
#criterion =  fnn.mse_loss
train_loss_min = 10000
val_loss_min = 10000
#batch = next(iter(train_dataloader))
for epoch in range(args.epochs):
    #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
    train_loss = train(model, train_dataloader, criterion, optimizer ,device)
    #if epoch % 500 == 0:
    print(train_loss)

    if (train_loss < train_loss_min):
        train_loss_min      = train_loss
        torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join(args.output_dir, "train.tgz")
            )  

    val_loss = evaluate(model, val_dataloader, criterion, device)
    # #break
    print(val_loss)

    print(calculate_predictions(model, val_dataloader))
    if (val_loss < val_loss_min):
         val_loss_min      = val_loss
         torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val.tgz"))

training...:   0%|          | 0/613 [00:00<?, ?it/s]

0.6767977554879803


validating...:   0%|          | 0/154 [00:00<?, ?it/s]

0.6981172343484716


predicting...:   0%|          | 0/154 [00:00<?, ?it/s]

...

0.5728294494004272


validating...:   0%|          | 0/154 [00:00<?, ?it/s]

0.739087588354653


predicting...:   0%|          | 0/154 [00:00<?, ?it/s]

0.5652826739783261
[[2460 2747]
 [3912 6199]]
              precision    recall  f1-score   support

           0       0.39      0.47      0.42      5207
           1       0.69      0.61      0.65     10111

    accuracy                           0.57     15318
   macro avg       0.54      0.54      0.54     15318
weighted avg       0.59      0.57      0.57     15318

(0.5652826739783261, 0.6505745920134334)


А теперь смотрим как отрабатывает классификатор на всех людях по раздельности

In [57]:
model_state  = torch.load(os.path.join(args.output_dir, f"val.tgz"))
  #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
model.load_state_dict(model_state['model_state_dict'])

<All keys matched successfully>

In [55]:
accs_all = []
f1_all = []
for sub in range(32):
  accs_all = []
  f1_all = []
  #for fold in range(len(ind_tests)):
  model_state  = torch.load(os.path.join(args.output_dir, f"val.tgz"))
  #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
  model.load_state_dict(model_state['model_state_dict'])
  val_dataset_sub = EmotionDataset(inds_test, data[sub:sub + 1], labels_bin[sub : sub + 1])
  val_dataloader_sub = DataLoader(val_dataset_sub, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
    
  #print(val_datar_sub.cnt)
  acc, f1 = calculate_predictions(model, val_dataloader_sub)
  accs_all.append(acc)
  f1_all.append(f1)
  # print(np.mean(accs_all))    
  # print(np.mean(f1_all))    
  # accs_result.append(np.mean(accs_all))
  # f1_result.append(np.mean(f1_all))


predicting...:   0%|          | 0/5 [00:00<?, ?it/s]

0.5138592750533049
[[168  61]
 [167  73]]
              precision    recall  f1-score   support

           0       0.50      0.73      0.60       229
           1       0.54      0.30      0.39       240

    accuracy                           0.51       469
   macro avg       0.52      0.52      0.49       469
weighted avg       0.52      0.51      0.49       469



predicting...:   0%|          | 0/5 [00:00<?, ?it/s]

0.5223880597014925
[[ 91  78]
 [146 154]]
              precision    recall  f1-score   support

           0       0.38      0.54      0.45       169
           1       0.66      0.51      0.58       300

    accuracy                           0.52       469
   macro avg       0.52      0.53      0.51       469
weighted avg       0.56      0.52      0.53       469



predicting...:   0%|          | 0/5 [00:00<?, ?it/s]

...

0.8507462686567164
[[  1  59]
 [ 11 398]]
              precision    recall  f1-score   support

           0       0.08      0.02      0.03        60
           1       0.87      0.97      0.92       409

    accuracy                           0.85       469
   macro avg       0.48      0.49      0.47       469
weighted avg       0.77      0.85      0.81       469



predicting...:   0%|          | 0/5 [00:00<?, ?it/s]

0.7654584221748401
[[  1 108]
 [  2 358]]
              precision    recall  f1-score   support

           0       0.33      0.01      0.02       109
           1       0.77      0.99      0.87       360

    accuracy                           0.77       469
   macro avg       0.55      0.50      0.44       469
weighted avg       0.67      0.77      0.67       469



predicting...:   0%|          | 0/5 [00:00<?, ?it/s]

0.464818763326226
[[ 58  51]
 [200 160]]
              precision    recall  f1-score   support

           0       0.22      0.53      0.32       109
           1       0.76      0.44      0.56       360

    accuracy                           0.46       469
   macro avg       0.49      0.49      0.44       469
weighted avg       0.63      0.46      0.50       469



In [56]:
print(np.mean(accs_all))    
print(np.mean(f1_all))   

0.464818763326226
0.5604203152364272
